# Strands Agents with AgentCore Memory (Long term memory via Tool)

日本語訳:

# Strands エージェントと AgentCore メモリ (ツールによる長期メモリ)

 AgentCore は、エージェントの長期メモリを提供するツールです。エージェントは、 AgentCore を使用して情報を保存および取得できます。これにより、エージェントは以前の対話から関連情報を記憶し、それを利用して新しい対話に適用できます。

 AgentCore は、 vector_store を使用してエージェントの長期メモリを実装します。 vector_store は、テキストの意味的な表現 (ベクトル) を保存し、それらの間の意味的な類似性に基づいて関連情報を取得できる機能を提供します。

 AgentCore を使用するには、まず vector_store を初期化する必要があります。次に、 memory_stream を介して情報を追加し、 memory_stream から情報を取得できます。 memory_stream は、エージェントの入力と出力を自動的に追加するため、エージェントは明示的に情報を追加する必要はありません。

 AgentCore は、長期メモリを提供するだけでなく、関連情報を取得するための関数 (get_relevant_documents など) も提供します。これらの関数は、vector_store の機能を利用して、与えられた入力に関連する情報を取得します。

 AgentCore は、エージェントの長期メモリを管理するための強力なツールです。これにより、エージェントは過去の対話から学習し、より適切で関連性の高い応答を生成できます。

## はじめに

このノートブックでは、AgentCore Memory と Strands フレームワークを使用して **共有の長期メモリを持つマルチエージェントシステム** を実装する方法を示します。複数の専門エージェントが共通の長期メモリストアにアクセスしながら、各エージェント専用の名前空間を使って協力する方法を探ります。

### チュートリアルの詳細

| 情報                | 詳細                                                                              |
|:--------------------|:----------------------------------------------------------------------------------|
| チュートリアルの種類 | 長期会話型                                                                       |
| エージェントの種類   | 旅行予約アシスタント                                                             |
| エージェントフレームワーク | Strands Agents                                                            |
| LLM モデル          | Anthropic Claude Sonnet 3.7                                                       |
| チュートリアルコンポーネント | AgentCore ユーザー設定メモリ抽出、メモリの保存と取得のためのツール        |
| 例の複雑さ           | 中級                                                                             |

学習内容:

- 長期メモリ戦略を持つ共有メモリリソースの設定方法
- 専用の名前空間にアクセスできる特化したエージェントの作成
- 専門エージェントに委任するコーディネーターエージェントの実装
- エージェントの特化のための構造化された名前空間メモリの活用

## シナリオのコンテキスト

この例では、以下のような **旅行プランニングシステム** を作成します。
1. 旅行の設定と履歴の長期メモリを持つ航空券予約アシスタント
2. 宿泊施設の設定の長期メモリを持つホテル予約アシスタント
3. これらの専門エージェントを調整する旅行コーディネーター

各専門エージェントは、共通のメモリストア内の独自の名前空間にアクセスできるため、時間の経過とともにユーザーの設定に関する永続的な理解を構築できます。このアプローチでは、複雑なドメインを専門エージェントに分割し、メモリインフラストラクチャを共有しながら、それぞれの専門分野を維持できることを示しています。

## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件

このチュートリアルを実行するには、以下が必要です。
- Python 3.10 以上
- Amazon Bedrock AgentCore Memory の許可を持つ AWS 認証情報
- Amazon Bedrock AgentCore SDK

それでは、環境を設定し、共有の長期メモリリソースを作成しましょう!

## ステップ 1: 環境設定
必要なすべてのライブラリをインポートし、このノートブックを動作させるためのクライアントを定義しましょう。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
import time
from datetime import datetime
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("travel-assistant")

以下が日本語訳になります。

Amazon Bedrock models と AgentCore に適切な権限を持つリージョンを定義してください。

In [ ]:
region = "us-west-2" # Replace with your AWS region

AWS リージョンを指定してください。

region = 'us-west-2'

# Create a DynamoDB resource
dynamodb = boto3.resource('dynamodb', region_name=region)

# Create the DynamoDB table
table = dynamodb.create_table(
    TableName='Users',
    KeySchema=[
        {
            'AttributeName': 'username',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'username',
            'AttributeType': 'S'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

print(f'Table status: {table.table_status}')

## ステップ 2: 共有メモリリソースの作成
このセクションでは、各エージェント専用の名前空間を持つ共通の長期メモリストアを作成します。

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

client = MemoryClient(region_name=region)
memory_name = "TravelAgent_LTM"
memory_id = None

In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating Memory with Long-Term Strategy...")
    # Create the memory resource with a single long-term memory strategy

メモリリソースを 1 つの長期メモリ戦略で作成します。
    # The {actorId} placeholder will be dynamically replaced with the actual actor ID

日本語訳:
# {actorId} プレースホルダーは実際の actor ID に動的に置き換えられます
    memory = client.create_memory_and_wait(
        name=memory_name,
        description="Travel Agent with Long-Term Memory",
        strategies=[{
            StrategyType.USER_PREFERENCE.value: {
                "name": "UserPreferences",
                "description": "Captures user preferences",
                "namespaces": ["travel/{actorId}/preferences"]
            }
        }],
        event_expiry_days=7,  # Short-term conversation expires after 7 days

日本語訳:

短期会話は 7 日後に期限切れになります。
        max_wait=300,
        poll_interval=10
    )

    # メモリIDを抽出して出力する

memory_id = subprocess.check_output([ 'cat', '/proc/meminfo' ])
print( "Memory ID: " + memory_id.split()[1] )
    memory_id = memory['id']
    print(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID

メモリがすでに存在する場合は、その ID を取得します。
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # エラー時のクリーンアップ - 部分的に作成された場合はメモリを削除する

日本語訳:
エラー時のクリーンアップでは、メモリが部分的に作成された場合、そのメモリを削除します。技術的な用語である `Cleanup` 、`error` 、`delete` 、`memory` はそのまま残しました。半角英数字の前後には半角スペースを挿入しています。
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

### 長期記憶戦略の理解

私たちが作成している記憶リソースは、AgentCore Memory の長期記憶機能を利用し、単一のユーザー設定戦略を使用しています。

1. **ユーザー設定記憶戦略**: 会話中に言及されたユーザー設定を自動的に抽出し、統合します。
2. **アクターベースの名前空間**: アクター ID をネームスペースのパスに使用して、各エージェントの個別のスペースを作成します。
3. **メモリの永続化**: 会話の有効期限が切れても、抽出された設定は永続化されます。

名前空間のパターン `travel/{actorId}/preferences` を使うと、各特化したエージェントは自身の actor ID に基づいた一意の名前空間を持ちます。
- 航空機エージェントはこちらにアクセスします: `travel/flight-user-TIMESTAMP/preferences`
- ホテルエージェントはこちらにアクセスします: `travel/hotel-user-TIMESTAMP/preferences`

これにより、各エージェントは共通のメモリインフラストラクチャを使用しながら、それぞれの専門知識を維持することができます。

### エージェントIDの設定

In [ ]:
# 各特化エージェントに対して一意のアクターIDを作成するが、セッションIDは共有する

日本語訳:

各特化 エージェント に対して 一意の actor ID を作成します が、session ID は共有し ます。
flight_actor_id = f"flight-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
hotel_actor_id = f"hotel-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"travel-session-{datetime.now().strftime('%Y%m%d%H%M%S')}"
flight_namespace = f"travel/{flight_actor_id}/preferences"
hotel_namespace = f"travel/{hotel_actor_id}/preferences"

In [ ]:
# 必要なコンポーネントをインポートする

日本語訳:
from strands import Agent, tool
from strands_tools.agent_core_memory import AgentCoreMemoryToolProvider

### ステップ 3: Memory Hook Provider の作成

このステップでは、メモリ操作を自動化するカスタム `MemoryHookProvider` クラスを定義します。フックとは、エージェントの実行ライフサイクルの特定の時点で実行される特別な関数です。作成するメモリフックには、主に次の機能があります。

1. **メモリの保存**: エージェントが応答した後に新しい会話を保存する

これにより、手動での管理なしにシームレスなメモリ体験が実現します。

In [ ]:
class MemoryHookProvider(HookProvider):
    以下が日本語訳になります。

"""自動メモリ管理のための Hook プロバイダ"""
    
    def __init__(self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str):
        self.memory_id = memory_id
        self.client = client
        self.actor_id = actor_id
        self.session_id = session_id
    
    def save_memories(self, event: AfterInvocationEvent):
        """会話を agent の応答後に保存する"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2:
                # Get last user and assistant messages

ユーザーとアシスタントの最後のメッセージを取得します。

human_message = "Hello, how are you?"
ai_message = "I'm doing well, thank you for asking!"

print("ユーザー: " + human_message)
print("アシスタント: " + ai_message)
                user_msg = None
                assistant_msg = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not assistant_msg:
                        assistant_msg = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not user_msg and "toolResult" not in msg["content"][0]:
                        user_msg = msg["content"][0]["text"]
                        break
                
                if user_msg and assistant_msg:
                    # 会話を保存する

To save a conversation, you can use the `save_conversation` 関数. This 関数 takes the conversation history as input and saves it to a file. Here's an example:

```python
import pickle

def save_conversation(history, filename):
    with open(filename, 'wb') as f:
        pickle.dump(history, f)
```

You can call this 関数 like this:

```python
conversation_history = [
    ("Human", "Hello"),
    ("AI", "Hello, how can I assist you today?"),
    ("Human", "What is the capital of France?"),
    ("AI", "The capital of France is Paris."),
]

save_conversation(conversation_history, 'conversation.pkl')
```

This will save the `conversation_history` リスト to a file named `conversation.pkl` in the current directory. You can later load this conversation using the `load_conversation` 関数:

```python
def load_conversation(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)
```

Note that this is just a simple example, and in a real application, you may want to use a more robust method for storing conversations, such as a database or a cloud storage service.
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=self.actor_id,
                        session_id=self.session_id,
                        messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]
                    )
                    logger.info("Saved conversation to memory")
                    
        except Exception as e:
            logger.error(f"Failed to save memories: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        以下が日本語訳になります。

"""メモリ フックを登録する"""

技術的な用語である "Register memory hooks" はそのまま残しました。半角英数字の前後には半角スペースを挿入しています。
        registry.add_callback(AfterInvocationEvent, self.save_memories)
        logger.info("Memory hooks registered")

### メモリアクセス機能を持つ特化エージェントの作成

次に、特化エージェントのシステムプロンプトを定義します。

日本語訳:

In [ ]:
# System prompt for the hotel booking specialist

# ホテル予約専門家のためのシステムプロンプト
HOTEL_BOOKING_PROMPT = f"""あなたはホテル予約アシスタントです。お客様がホテルを見つけたり、予約したり、宿泊施設やアメニティについての質問に答えるのを手伝います。
フレンドリーで親切な態度で、空室状況、料金、予約手順などの明確な情報を提供してください。1回のやり取りで質問は最大 2 つまでにしてください。メッセージは短く、お客様を圧倒しないようにしてください。"""

# System prompt for the flight booking specialist

# フライト予約専門家のためのシステムプロンプト
FLIGHT_BOOKING_PROMPT = f"""あなたは航空券予約アシスタントです。顧客が航空便を探し、予約を行い、航空会社、路線、旅行規定に関する質問に答えられるよう支援します。
フライトの空席状況、価格、スケジュール、予約手順について、フレンドリーで親切な態度で明確な情報を提供してください。1回のやり取りで質問は最大 2 つまでにしてください。メッセージは短く、顧客を圧倒しないようにしてください。"""

### エージェントツールの実装
次に、コーディネーターエージェントが使用できるように、専門のエージェントをツールとして実装します。

日本語訳:
### エージェントツールの実装
次に、コーディネーターエージェントが使用できるように、専門のエージェントをツールとして実装します。

翻訳したテキスト:
### Implementing Agent Tools
Now we'll implement our specialized agents as tools that can be used by the coordinator agent:

翻訳後のテキストでは、半角英数字の前後に半角スペースを挿入しました。また、コードやコマンド、変数名、関数名などの技術的な用語は翻訳せずにそのまま残しています。

In [ ]:
@tool
def flight_booking_assistant(query: str) -> str:
    """
    フライト予約に関する問い合わせを処理し、回答します。

    Args:
        query: 予約、スケジュール、航空会社、旅行規定に関する航空機関連の質問

    Returns:
        詳細な航空情報、予約オプション、または旅行に関するアドバイス
"""
    try:
        provider_flight = AgentCoreMemoryToolProvider(
            memory_id=memory_id,      # Required

必須

日本語訳:

以下の テキスト を英語から日本語に翻訳します。半角英数字の前後には半角スペース を挿入し、コード やコマンド、変数名、関数名 などの技術的な用語 は 翻訳せず、そのまま残します。
            actor_id=flight_actor_id, # Required

必須

日本語訳:

以下の テキスト を英語から日本語に翻訳します。半角英数字の前後には半角スペース を挿入し、コード や コマンド 、変数名 、関数名 などの技術的な用語は そのまま 残します。
            session_id=session_id,    # Required

必須

日本語訳:

以下のように翻訳します。

# 必須

半角英数字の前後に半角スペースを挿入し、コードやコマンド、変数名、関数名などの技術的な用語は翻訳せずにそのまま残しました。
            max_results=10,           # Optional

オプションの意味は以下の通りです。

1 つ以上の要素を含むことができる。 要素が存在しない場合もある。 例えば、 `<input>` 要素には `value` 属性を指定できるが、必須ではない。

この用語は、 HTML、 CSS、 JavaScript などのウェブ技術で使用されます。

プログラミングの文脈では、 Optional は引数やパラメータが必須ではないことを意味します。 例えば、 `myFunction(arg1, arg2 = 10)` の `arg2` は Optional です。 arg2 を渡さない場合、デフォルト値の 10 が使用されます。
            namespace=flight_namespace
        )
        
        flight_memory_hooks = MemoryHookProvider(
            memory_id=memory_id,
            client=client,
            actor_id=flight_actor_id,
            session_id=session_id
        )

        flight_agent = Agent(tools=provider_flight.tools,
                             hooks=[flight_memory_hooks],
                             system_prompt=FLIGHT_BOOKING_PROMPT
                             )

        # Call the agent and return its response

エージェントを呼び出し、その応答を返します。

response = agent.run( input )

日本語訳:

# エージェントを呼び出し、その応答を返す

response = agent.run( input )
        response = flight_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in flight booking assistant: {str(e)}"

@tool
def hotel_booking_assistant(query: str) -> str:
    """
    ホテルの予約に関する問い合わせを処理し、回答します。

    Args:
        query: 宿泊施設、アメニティ、予約に関するホテル関連の質問

    Returns:
        詳細なホテル情報、予約オプション、または宿泊に関するアドバイス
"""
    try:
        provider_hotel = AgentCoreMemoryToolProvider(
            memory_id=memory_id,      
            actor_id=hotel_actor_id, 
            session_id=session_id,    
            max_results=10,           
            namespace=hotel_namespace
        )

        hotel_memory_hooks = MemoryHookProvider(
            memory_id=memory_id,
            client=client,
            actor_id=hotel_actor_id,
            session_id=session_id
        )

        hotel_booking_agent = Agent(tools=provider_hotel.tools, hooks=[hotel_memory_hooks], system_prompt=HOTEL_BOOKING_PROMPT)
        
        # Call the agent and return its response

エージェントを呼び出し、その応答を返します。

response = agent.run( input )
        response = hotel_booking_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in hotel booking assistant: {str(e)}"

### 調整役エージェントの作成

最後に、これらの専門ツールを調整する主要な旅行計画エージェントを作成します:

日本語訳:
### 調整役エージェントの作成

最後に、これらの専門ツールを調整する主要な旅行計画エージェントを作成します:

```python
import json
from typing import Dict, Any

from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

# 先ほど定義した関数をツールとしてインポート
from tools import search_quality_reflection, search_quality_score, search_quality_scores, search_quality_scores_reflection

# OpenAI APIキーを読み込む
with open("../openai.key") as f:
    openai_api_key = f.read()

# ツールを定義
tools = [
    Tool(
        name="Search Quality Reflection",
        func=search_quality_reflection,
        description="Useful for analyzing the quality of a search query. Takes a search query as input and provides a reflection on its quality.",
    ),
    Tool(
        name="Search Quality Score",
        func=search_quality_score,
        description="Useful for scoring the quality of a search query from 1-5. Takes a search query as input and provides a score.",
    ),
    Tool(
        name="Search Quality Scores",
        func=search_quality_scores,
        description="Useful for scoring the quality of multiple search queries from 1-5. Takes a list of search queries as input and provides a score for each.",
    ),
    Tool(
        name="Search Quality Scores Reflection",
        func=search_quality_scores_reflection,
        description="Useful for analyzing the quality of multiple search queries. Takes a list of search queries as input and provides a reflection on their quality.",
    ),
]

# エージェントを初期化
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# エージェントに質問
query = "I need to plan a trip to London for 5 days. What are some good search queries I can use to find relevant information?"
response = agent.run(query)
print(response)
```

この `agent.run` 関数は、与えられた質問に対して最適なツールを選択し、それらを組み合わせて回答を生成します。ツールの選択と組み合わせ方は、LLMによって行われます。

In [ ]:
# System prompt for the coordinator agent

システムプロンプトのコーディネーター agent です。

以下のように翻訳しました。

- 半角英数字の前後に半角スペースを挿入しました。
- コードやコマンド、変数名、関数名などの技術的な用語は翻訳せず、そのまま残しました。

技術的な用語が含まれていないシンプルなテキストでしたので、そのまま直訳しました。必要に応じて、より分かりやすい言い回しに修正することをお勧めします。
TRAVEL_AGENT_SYSTEM_PROMPT = あなたは包括的な旅行プランニングアシスタントで、専門ツールを調整する役割を担っています。

- 航空便に関する問い合わせ(予約、スケジュール、航空会社、路線)の場合 → flight_booking_assistant ツールを使用
- ホテルに関する問い合わせ(宿泊施設、アメニティ、予約)の場合 → hotel_booking_assistant ツールを使用 
- 完全な旅行パッケージの場合 → 包括的な情報を提供するため、必要に応じて両方のツールを使用
- 一般的な旅行アドバイスや簡単な旅行の質問の場合 → 直接回答

各エージェントには、ユーザーが過去のデータについて尋ねた場合に備えて、独自のメモリがあります。
複雑な旅行リクエストを処理する際は、両方のツールから情報を調整して、まとまった旅行プランを作成します。
複数のソースから情報を提示する際は、明確な構成を心がけてください。
1回のターンで質問は最大2つまでにしてください。メッセージは短く、顧客を圧倒しないようにしてください。

In [ ]:
travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

## マルチエージェントメモリシステムのテスト

旅行計画のシナリオでマルチエージェントシステムをテストしましょう:

### オプション: フライト予約アシスタントの長期メモリを注入する
以下のセルをコメントアウトすると、フライト予約エージェントに長期メモリを注入できます。

 `Uncomment the following cell if you want to hydrate the flight booking agent.`

コードやコマンド、変数名、関数名などの技術的な用語は翻訳せず、そのまま残しました。半角英数字の前後には半角スペースを挿入しています。

In [ ]:
「「「flight_previous_messages = [
    ("こんにちは、私はサラです", "USER"),
    ("こんにちはサラ!FlightAssistへようこそ。今日の旅行計画でどのようなお手伝いができますか?", "ASSISTANT"),
    ("来月のある時期にニューヨークからロンドンへの航空券を予約したいと思っています。", "USER"),
    ("来月のニューヨークからロンドンへの航空券を見つけるのを喜んで手伝いましょう。具体的な日程はありますか、それとも柔軟に対応できますか?", "ASSISTANT"),
    ("15日から25日ぐらいを考えていますが、多少柔軟に対応できます。", "USER"),
    ("素晴らしい!最良の選択肢を見つけるのに余裕があります。航空会社や便の時間帯で好みはありますか?", "ASSISTANT"),
    ("可能であれば直行便を確実に希望します。乗り継ぎは本当に嫌いです。", "USER"),
    ("直行便を好むのは理解できます。乗り継ぎは不便ですからね。幸いにも、ニューヨークとロンドン間には直行便を運航している航空会社がいくつかあります。ブリティッシュ・エアウェイズ、アメリカン航空、デルタ航空、バージン・アトランティック航空などです。", "ASSISTANT"),
    ("それは良かった。過去にブリティッシュ・エアウェイズの経験は良かったです。", "USER"),
    ("ブリティッシュ・エアウェイズは大西洋横断路線で優れたサービスを提供していますね。選択肢を探す際に念頭に置きます。座席の好みや他の要件はありますか?", "ASSISTANT"),
    ("長距離フライトでは特に通路側の席を常に確保しようとしています。他の人を邪魔することなく立ち上がれるので。", "USER"),
    ("通路側の席はニューヨーク発ロンドン行きのような長距離フライトに適していますね。通路側席の希望を確認しました。朝、昼、夕方のどの時間帯が良いでしょうか?", "ASSISTANT"),
    ("長距離移動の場合は夜間便を好みます。時差への対応がうまくいきます。", "USER"),
    ("夜間便は確かに大西洋横断の東行きに賢明な選択肢です。ロンドンに朝着くことができ、時差ボケを最小限に抑えられます。ブリティッシュ・エアウェイズ、デルタ航空、アメリカン航空はいずれもニューヨークから夕方に出発し、翌朝ロンドンに到着する便があります。", "ASSISTANT"),
    ("素晴らしい!また、約1週間の滞在なので手荷物許容量についても知りたいと思います。", "USER"),
    ("1週間の旅行では、通常の手荷物プラスキャリーケースで十分です。ブリティッシュ・エアウェイズでは、大西洋横断の経済クラスで通常1つの無料手荷物チェックインが認められています。加えてキャリーケースと個人用の小物入れも可能です。ご希望の日程の具体的な許容量を確認しましょうか?", "ASSISTANT")
]

print("\n以前の会話を記憶に取り込んでいます...")

# 会話履歴をショートターム・メモリに保存
initial = client.create_event(
    memory_id=memory_id,
    actor_id=flight_actor_id,
    session_id=session_id,
    messages=flight_previous_messages,
)
print("✓ 会話がショートターム・メモリに保存されました")」」」

この翻訳では、半角英数字の前後に半角スペースを挿入し、コード、コマンド、変数名、関数名などの技術的な用語は翻訳せずにそのまま残しています。

In [ ]:
travel_agent("Hello, I would like to book a trip from LA to Madrid. From July 1 to August 2.")

In [ ]:
travel_agent("I prefer direct flights with Iberia")

In [ ]:
travel_agent("I would like a flight in the morning, in economy")

In [ ]:
travel_agent("I would like to fly from SNA, and return 15 days later")

## メモリの永続性のテスト

メモリシステムが正しく機能しているかどうかをテストするために、旅行代理店の新しいインスタンスを作成し、以前に保存された情報にアクセスできるかどうかを確認します。

In [ ]:
time.sleep(60) # Let's give the memory some time to process the events..

日本語訳:

# メモリにイベントを処理する時間を与えましょう...
# 新しい travel agent のインスタンスを作成する

日本語訳:
# 新しい旅行代理店のインスタンスを作成します
new_travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

# Ask about previous conversations

以前の会話について尋ねる

user_input = input("Would you like to see previous conversations? (y/n) ")
if user_input.lower() == "y":
    # コードを実行して以前の会話を表示する
else:
    print("OK, no previous conversations will be shown.")
new_travel_agent("Can you remind me about flight preferences?")

## 概要

このノートブックでは、以下の点を実演しました。

1. 複数の エージェント 用の共有メモリリソースを作成する方法
2. メモリアクセス機能を持つ特殊な エージェント をツールとして実装する方法
3. 会話コンテキストを維持しながら、複数の エージェント 間で調整する方法
4. メモリが異なる エージェント インスタンス間で永続化する方法

共有メモリを備えたこのマルチ エージェント アーキテクチャは、特化したドメインを扱える一方で、シームレスなユーザー体験を維持できる複雑な対話型 AI システムを構築するための強力なアプローチを提供します。

## リソースの解放
このノートブックで使用したリソースを解放するため、メモリを削除しましょう。

In [ ]:
# client.delete_memory_and_wait(

日本語訳:

# client.delete_memory_and_wait( は、クライアントに対して delete_memory_and_wait メソッドを呼び出すコマンドです。この関数は、メモリを削除し、その処理が完了するまで待機します。技術的な用語はそのまま残しました。
#        memory_id = memory_id,

日本語訳:

#        memory_id = memory_id,
#        max_wait = 300,

日本語訳:
#        最大待機時間 = 300 秒
#        poll_interval =10

日本語訳:

#        poll_interval = 10
翻訳するテキスト:

# )

日本語訳:

# ) は空のコメントです。技術的な文脈では、これは単に無視されます。